In [1]:
## Simultaneous Domain Adaptation

In [27]:
import requests
import os
import numpy as np
from collections import Counter, defaultdict
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
import torchmetrics
from torchmetrics import MetricCollection, MeanMetric, Accuracy, F1Score, Precision, Recall, AUROC, ConfusionMatrix
import tensorboard
import torch.optim as optim
from tqdm.notebook import tqdm 
import seaborn as sns
import torchvision.models as models

from dataclasses import dataclass, field
from typing import Dict, Tuple, Optional, Any
from pathlib import Path
from PIL import Image

In [28]:
# The image size of the Binsity is 640x480
train_transform = T.Compose([
    T.CenterCrop((480, 480)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(degrees = 15),
    T.ColorJitter(brightness = 0.2, contrast = 0.2, saturation = 0.2, hue = 0.1),        

    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# train_transform = T.Compose([
#     T.CenterCrop((480, 480)),
    
#     # --- GEOMETRIC AUGMENTATIONS ---
#     T.RandomHorizontalFlip(p=0.5),
#     T.RandomRotation(degrees=15), 
#     T.RandomPerspective(distortion_scale=0.3, p=0.4), # Reduced p for perspective
    
#     # --- CONSERVATIVE COLOR/NOISE AUGMENTATIONS ---
#     # Reduced range to account for minor noise/sensor variation, not environment change
#     T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    
#     # --- RESOLUTION & NORMALIZATION (448x448) ---
#     T.Resize((224, 224)), # <-- NEW TARGET RESOLUTION
#     T.ToTensor(),
#     T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    
#     # --- OCCLUSION AUGMENTATION (After normalization) ---
#     # Simulates small occlusions/stickers, which is great for generalization
#     T.RandomErasing(p=0.4, scale=(0.02, 0.15), value='random') 
#     # Reduced scale max from 0.2 to 0.15 to prevent erasing too much detail
# ])

eval_transform = T.Compose([
    # Pad 80 pixels above and 80 below
    T.CenterCrop((480, 480)),
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

binsity_datapath = 'data/binsity/TrashNetDataset'
trashnet_datpath = 'data/dataset-resized/'



In [29]:
class BinscityDataset(Dataset):
    def __init__(self, dataset_path, transform = None):
        super().__init__()
        self.dataset_path = Path(dataset_path)
        self.transform = transform
        self.class_names = [path.name for path in sorted(list(self.dataset_path.glob("*")))]
        
        self.idx2label = {idx: label for idx, label in enumerate(self.class_names)}
        self.label2idx = {label: idx for idx, label in self.idx2label.items()}

        # Need to group all of the files by Group (class name) first to 
        # perform the per-class sampling protocol
        self.files_by_class = defaultdict(list)
        all_files = sorted(list(self.dataset_path.glob("*/*.*")))

        # Perform Protocol partitioning
        self.final_paths = []
        self.final_labels = []

        for img_path in all_files:
            class_name = img_path.parent.name
            self.final_paths.append(img_path)
            self.final_labels.append(self.label2idx[class_name])
    
    def get_labels(self):
        return self.idx2label, self.label2idx
    
    def __len__(self):
        return len(self.final_paths)
    
    def __getitem__(self, idx):
        path = self.final_paths[idx]
        label = self.final_labels[idx]

        try:
            image = Image.open(path).convert('RGB')
        except Exception as e:
            print(f"Unexpected error has founded: {e}")
            return self.__getitem__((idx + 1) % len(self))

        if self.transform:
            image = self.transform(image)
        else:
            image = torch.as_tensor(np.array(image)).permute(2, 0, 1).float() / 255.0

        return image, label

In [30]:
binsity_dataset = BinscityDataset(binsity_datapath)

first_instance  = binsity_dataset[0]
image, label = first_instance

print(f"Image shape: {image.shape}")
print(f"label is: {label}")

idx2label, label2idx = binsity_dataset.get_labels()
num_classes = len(idx2label)

Image shape: torch.Size([3, 480, 640])
label is: 0


## Function routine

In [31]:
def getting_models(checkpoint_path, num_classes = 6, device = "cpu"):
    # Load the MobileNetV3_Small model
    model = models.mobilenet_v3_small()

    # Modify the classifier to match the number of our output classes
    model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)

    # Load best model for prediction
    if os.path.exists(checkpoint_path):
        model.load_state_dict(torch.load(checkpoint_path))
        print("Loaded best model for prediction")

    model.to(device)
    model.eval()

    return model

checkpoint_path = "checkpoints/best_model_v2.pth"
device = "cuda:0"
model = getting_models(checkpoint_path, num_classes, device)


Loaded best model for prediction


In [32]:
from tqdm.notebook import tqdm

# --- Helper Functions (Training and Validation Epochs) ---
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    
    # Initialize tqdm for the training loader
    # desc='Training' sets the label, leave=False clears the bar on completion
    train_bar = tqdm(loader, desc='Training', leave=False)
    
    for inputs, labels in train_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        
        # Update the tqdm bar with the current batch loss
        train_bar.set_postfix({'batch_loss': f'{loss.item():.4f}'})
        
    return running_loss / len(loader.dataset)

@torch.no_grad()
def validate_epoch(model, loader, criterion, device, num_classes=6):
    model.eval()
    running_loss = 0.0

    metrics = MetricCollection({
        'Accuracy': Accuracy(task="multiclass", num_classes=num_classes),
        'F1Score': F1Score(task="multiclass", num_classes=num_classes),
        'Precision': Precision(task="multiclass", num_classes=num_classes),
        'Recall': Recall(task="multiclass", num_classes=num_classes),
        'AUROC': AUROC(task="multiclass", num_classes=num_classes)
    }).to(device)

    # Initialize tqdm for the validation loader
    # desc='Validation' sets the label, leave=False clears the bar on completion
    val_bar = tqdm(loader, desc='Validation', leave=False)

    for inputs, labels in val_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        running_loss += loss.item() * inputs.size(0)
        
        metrics.update(outputs, labels)
        
        val_bar.set_postfix({'batch_loss': f'{loss.item():.4f}'})
        
    val_loss = running_loss / len(loader.dataset)
    
    final_metrics = metrics.compute()
    
    metrics.reset() 
    
    return val_loss, final_metrics

This step is combating the data imbalanced problem
by assigning different weights to each label when calculating the loss
function.

- Meaning: giving more emphasis (more weight) on the label with less instances 

In [33]:
def get_label_counts(dataset, label_names, device = "cpu"):
    label_counts = Counter([label_names[labels] for _, labels in dataset])
    
    total_weight = sum(label_counts.values())
    class_weights = [total_weight / label_counts[i] for i in label_counts.keys()]
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)
    return class_weights_tensor

def get_label_counts_softened(dataset, label_names, device="cpu"):
    """
    Calculates class weights using the square-root inverse frequency (softening)
    to prevent rare classes from dominating the loss function.
    """
    
    # 1. Get raw label counts
    label_counts = Counter([labels for _, labels in dataset])
    
    # Ensure all classes are present (if labels are indices 0 to N-1)
    num_classes = len(label_names)
    counts_list = [label_counts.get(i, 1) for i in range(num_classes)] # Use 1 if count is 0
    
    # 2. Calculate Inverse Frequency
    total_samples = sum(counts_list)
    inverse_frequency = [total_samples / count for count in counts_list]
    
    # 3. Apply Square Root Scaling (Softening)
    # This prevents the largest weights from dominating excessively.
    soft_weights = np.sqrt(inverse_frequency)
    
    # 4. Convert to Tensor and normalize (optional but good practice)
    class_weights_tensor = torch.tensor(soft_weights, dtype=torch.float)
    
    # Normalize so the average weight is 1 (prevents large loss magnitude)
    mean_weight = class_weights_tensor.mean()
    class_weights_tensor = class_weights_tensor / mean_weight
    
    return class_weights_tensor.to(device)

label_names, _ = binsity_dataset.get_labels()
class_weight_tensor = get_label_counts(binsity_dataset, label_names, device = device)
class_weight_softed_tensor = get_label_counts_softened(binsity_dataset, label_names, device = device)

print(class_weight_tensor)
print(class_weight_softed_tensor)


tensor([12.8387,  6.2188,  7.5094,  5.7681,  3.4017,  6.2188], device='cuda:0')
tensor([1.3819, 0.9618, 1.0569, 0.9263, 0.7113, 0.9618], device='cuda:0')


Performing K-Fold Cross Vallidation (KF = 5 fold)

In [34]:
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset
import copy

In [20]:
def train_and_validate_kfold(
    dataset,
    model_init_fn,
    num_classes,
    num_folds=5,
    epochs_frozen=10,
    lr_frozen=1e-3,
    epochs_unfrozen=10,
    lr_unfrozen=1e-5,
    batch_size=32,
    device="cuda:0",
    weight_path = "checkpoints/best_model_v2.pth", 
    checkpoint_dir="kfold_checkpoints",
    **kwargs
):
    """
    Performs K-Fold Cross-Validation with two-stage finetuning.
    
    Args:
        dataset (Dataset): The full dataset instance (e.g., BinscityDataset).
        model_init_fn (function): A function to initialize a fresh model (e.g., getting_models).
        num_classes (int): The number of output classes.
        ... (other parameters)
    """
    
    # 1. Setup
    os.makedirs(checkpoint_dir, exist_ok=True)
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    
    # Store results including all metrics
    all_results = [] 
    
    # Get all indices from the dataset
    all_indices = list(range(len(dataset)))

    print(f"Starting {num_folds}-Fold Cross-Validation...")
    
    # --- K-Fold Loop ---
    for fold, (train_indices, val_indices) in enumerate(kfold.split(all_indices)):
        print(f"\n{'='*30} FOLD {fold+1}/{num_folds} {'='*30}")
        
        # 2. Data Preparation for the current fold
        train_subset = Subset(dataset, train_indices)
        val_subset = Subset(dataset, val_indices)
                
        # Create DataLoaders
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=4)
        
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=4)
        # 3. Model Initialization
        model = model_init_fn(weight_path, num_classes=num_classes, device=device)
        
        # Check if class_weight_tensor is provided and convert to device tensor
        class_weight = kwargs.get('class_weight_tensor', None)
        if class_weight is not None and isinstance(class_weight, torch.Tensor):
            class_weight = class_weight.to(device)

        criterion = nn.CrossEntropyLoss(
            weight = class_weight,
            label_smoothing = kwargs.get('label_smoothing', 0.0) # Safe access with default
        )
        
        best_val_loss = float('inf')
        best_fold_model_wts = copy.deepcopy(model.state_dict())

        # --- Stage 1: Frozen Features (Finetune Classifier) ---
        print("\n--- Stage 1: Frozen Features (10 Epochs, LR=1e-3) ---")
        
        # Freeze all layers except the classifier
        for name, param in model.named_parameters():
            if 'classifier' not in name:
                param.requires_grad = False
            else:
                param.requires_grad = True
        
        # Optimizer only for trainable parameters
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr_frozen)
        
        # Training/Validation Loop (Stage 1)
        for epoch in range(epochs_frozen):
            dataset.transform = train_transform
            train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
            
            dataset.transform = eval_transform
            # Pass num_classes to validate_epoch
            val_loss, final_metrics = validate_epoch(model, val_loader, criterion, device, num_classes=num_classes)
            val_acc = final_metrics['Accuracy'].item() # Extract accuracy for logging
            
            print(
                f"Epoch {epoch+1}/{epochs_frozen} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
                f"Val Acc: {val_acc:.4f} | F1: {final_metrics['F1Score'].item():.4f}"
            )
            
            # Save the best model weights based on validation loss
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_fold_model_wts = copy.deepcopy(model.state_dict())

        # --- Stage 2: Unfrozen Features (Finetune All) ---
        print("\n--- Stage 2: Unfrozen Features (10 Epochs, LR=1e-5) ---")
        
        # Unfreeze all layers
        for param in model.parameters():
            param.requires_grad = True

        # Re-initialize optimizer for ALL parameters with slower LR
        optimizer = optim.Adam(model.parameters(), lr=lr_unfrozen) 
        
        # Training/Validation Loop (Stage 2)
        for epoch in range(epochs_unfrozen):
            dataset.transform = train_transform
            train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
            
            dataset.transform = eval_transform
            # Pass num_classes to validate_epoch
            val_loss, final_metrics = validate_epoch(model, val_loader, criterion, device, num_classes=num_classes)
            val_acc = final_metrics['Accuracy'].item()
            
            print(
                f"Epoch {epoch+1+epochs_frozen}/{epochs_frozen+epochs_unfrozen} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
                f"Val Acc: {val_acc:.4f} | F1: {final_metrics['F1Score'].item():.4f}"
            )

            # Save the best model weights across both stages
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_fold_model_wts = copy.deepcopy(model.state_dict())

        # --- 4. Finalizing Fold ---
        
        # Load the best weights found across both stages for this fold
        model.load_state_dict(best_fold_model_wts)
        
        # Evaluate final performance on the fold's validation set
        dataset.transform = eval_transform
        final_val_loss, final_val_metrics = validate_epoch(model, val_loader, criterion, device, num_classes=num_classes)
        final_val_acc = final_val_metrics['Accuracy'].item()
        
        # Format metrics for clean output
        metric_str = ", ".join([f"{k}: {v.item():.4f}" for k, v in final_val_metrics.items()])
        
        print(f"\nFold {fold+1} Final Metrics | Loss: {final_val_loss:.4f} | {metric_str}")
        
        # Store results
        all_results.append({
            'fold': fold + 1,
            'val_loss': final_val_loss, 
            'metrics': {k: v.item() for k, v in final_val_metrics.items()}
        })
        
        # Save the best model for this fold
        torch.save(best_fold_model_wts, os.path.join(checkpoint_dir, f'best_model_fold_{fold+1}.pth'))

    # --- 5. Overall Results ---
    
    # Calculate averages from all folds
    avg_val_loss = sum(r['val_loss'] for r in all_results) / num_folds
    
    # Calculate average for each metric
    metric_keys = all_results[0]['metrics'].keys() if all_results else []
    avg_metrics = {}
    for key in metric_keys:
        avg_metrics[key] = sum(r['metrics'][key] for r in all_results) / num_folds

    avg_metric_str = ", ".join([f"Avg {k}: {v:.4f}" for k, v in avg_metrics.items()])
    
    print("\n" + "="*70)
    print(f"** Final K-Fold Results (Average over {num_folds} folds) **")
    print(f"Average Validation Loss: {avg_val_loss:.4f}")
    print(avg_metric_str)
    print("="*70)
    
    return all_results

all_results = train_and_validate_kfold(binsity_dataset, getting_models, num_classes,
                                       class_weight_tensor = class_weight_softed_tensor,
                                       label_smoothing = 0.1)

Starting 5-Fold Cross-Validation...

============================== FOLD 1/5 ==============================
Loaded best model for prediction

--- Stage 1: Frozen Features (10 Epochs, LR=1e-3) ---


Training:   0%|          | 0/10 [00:00<?, ?it/s]

Validation:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/10 | Train Loss: 1.6082 | Val Loss: 1.3224 | Val Acc: 0.5125 | F1: 0.5125


Training:   0%|          | 0/10 [00:00<?, ?it/s]

...

Training:   0%|          | 0/10 [00:00<?, ?it/s]

Validation:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch 20/20 | Train Loss: 0.7226 | Val Loss: 0.7983 | Val Acc: 0.8228 | F1: 0.8228


Validation:   0%|          | 0/3 [00:00<?, ?it/s]


Fold 5 Final Metrics | Loss: 0.7983 | AUROC: 0.9858, Accuracy: 0.8228, F1Score: 0.8228, Precision: 0.8228, Recall: 0.8228

** Final K-Fold Results (Average over 5 folds) **
Average Validation Loss: 0.8278
Avg AUROC: 0.9799, Avg Accuracy: 0.8342, Avg F1Score: 0.8342, Avg Precision: 0.8342, Avg Recall: 0.8342


## Retraining the dataset and take the result

In [35]:
def final_retrain_deployment_model(
    dataset,
    model_init_fn,
    num_classes,
    epochs_frozen=10,
    lr_frozen=1e-3,
    epochs_unfrozen=10,
    lr_unfrozen=1e-5,
    batch_size=32,
    device="cuda:0",
    weight_path="checkpoints/best_model_v2.pth",
    final_checkpoint_path="checkpoints/best_model_finetune.pth",
    **kwargs
):
    """
    Trains the final deployment model using the entire dataset with the 
    two-stage finetuning protocol.

    Args:
        dataset (Dataset): The full dataset instance.
        model_init_fn (function): Function to initialize a fresh model.
        ... (other hyperparameters)
        final_checkpoint_path (str): Path to save the final weights.
    """
    
    print("="*70)
    print("🚀 Starting Final Deployment Model Retraining on FULL Dataset 🚀")
    print("="*70)

    # 1. Data Loader (Use ALL data as the single training set)
    # The entire dataset is passed directly to the DataLoader
    dataset.transform = train_transform # Ensure train transform is set initially
    full_train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    
    # 2. Model Initialization
    # Initialize a fresh model weights using the base pre-trained weights
    model = model_init_fn(weight_path, num_classes=num_classes, device=device)
    
    # Check if class_weight_tensor is provided and convert to device tensor
    class_weight = kwargs.get('class_weight_tensor', None)
    if class_weight is not None and isinstance(class_weight, torch.Tensor):
        class_weight = class_weight.to(device)

    criterion = nn.CrossEntropyLoss(
        weight = class_weight,
        label_smoothing = kwargs.get('label_smoothing', 0.0)
    )
    
    # We track the best weights, even though there's no separate validation set
    # A common practice is to save the model based on lowest training loss 
    # or simply save the final epoch's weights, but tracking loss is safer.
    best_train_loss = float('inf')
    best_model_wts = copy.deepcopy(model.state_dict())

    # --- Stage 1: Frozen Features (Finetune Classifier) ---
    print(f"\n--- Stage 1: Frozen Features ({epochs_frozen} Epochs, LR={lr_frozen}) ---")
    
    # Freeze all layers except the classifier
    for name, param in model.named_parameters():
        if 'classifier' not in name:
            param.requires_grad = False
        else:
            param.requires_grad = True
    
    # Optimizer only for trainable parameters (classifier)
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr_frozen)
    
    # Training Loop (Stage 1)
    for epoch in range(epochs_frozen):
        train_loss = train_epoch(model, full_train_loader, criterion, optimizer, device)
        
        print(f"Stage 1 | Epoch {epoch+1}/{epochs_frozen} | Train Loss: {train_loss:.4f}")
        
        # Save best weights based on lowest training loss for this stage
        if train_loss < best_train_loss:
            best_train_loss = train_loss
            best_model_wts = copy.deepcopy(model.state_dict())

    # --- Stage 2: Unfrozen Features (Finetune All) ---
    print(f"\n--- Stage 2: Unfrozen Features ({epochs_unfrozen} Epochs, LR={lr_unfrozen}) ---")
    
    # Unfreeze all layers
    for param in model.parameters():
        param.requires_grad = True

    # Re-initialize optimizer for ALL parameters with slower LR
    optimizer = optim.Adam(model.parameters(), lr=lr_unfrozen) 
    
    # Training Loop (Stage 2)
    for epoch in range(epochs_unfrozen):
        train_loss = train_epoch(model, full_train_loader, criterion, optimizer, device)
        
        print(f"Stage 2 | Epoch {epoch+1+epochs_frozen}/{epochs_frozen+epochs_unfrozen} | Train Loss: {train_loss:.4f}")

        # Save best weights across both stages
        if train_loss < best_train_loss:
            best_train_loss = train_loss
            best_model_wts = copy.deepcopy(model.state_dict())

    # --- 3. Final Export ---
    
    # Load the absolute best weights found during the full training run
    model.load_state_dict(best_model_wts)
    
    # Ensure the checkpoint directory exists
    os.makedirs(os.path.dirname(final_checkpoint_path), exist_ok=True)
    
    # Save the final model weights
    torch.save(best_model_wts, final_checkpoint_path)
    
    print("\n" + "="*70)
    print(f"✅ Training Complete. Final Model saved to: {final_checkpoint_path}")
    print(f"Final Best Training Loss: {best_train_loss:.4f}")
    print("="*70)

In [36]:
# Remember to put the label_smoothing to 0.05 
# However it cannot perform well with Revive bottle :))) 

final_retrain_deployment_model(binsity_dataset, getting_models, num_classes,
                                       class_weight_tensor = class_weight_softed_tensor,
                                       epochs_frozen=10,  lr_frozen=1e-3,
                                       epochs_unfrozen=10, lr_unfrozen=5e-5,
                                       label_smoothing = 0.075)

🚀 Starting Final Deployment Model Retraining on FULL Dataset 🚀
Loaded best model for prediction

--- Stage 1: Frozen Features (10 Epochs, LR=0.001) ---


Training:   0%|          | 0/13 [00:00<?, ?it/s]

Stage 1 | Epoch 1/10 | Train Loss: 1.3938


Training:   0%|          | 0/13 [00:00<?, ?it/s]

Stage 1 | Epoch 2/10 | Train Loss: 1.0293


...

Stage 2 | Epoch 18/20 | Train Loss: 0.4939


Training:   0%|          | 0/13 [00:00<?, ?it/s]

Stage 2 | Epoch 19/20 | Train Loss: 0.4820


Training:   0%|          | 0/13 [00:00<?, ?it/s]

Stage 2 | Epoch 20/20 | Train Loss: 0.4797

✅ Training Complete. Final Model saved to: checkpoints/best_model_finetune.pth
Final Best Training Loss: 0.4797
